In [1]:
%matplotlib qt
import src
from src.simul_sweep import SimulSweep
from src.daq_driver import Daq
from src.base_sweep import BaseSweep
from src.sweep0d import Sweep0D
from src.sweep1d import Sweep1D
from src.sweep2d import Sweep2D
from src.sweep_ips import SweepIPS
from src.sweep_queue import SweepQueue, DatabaseEntry
from src.util import init_database
from src.tracking import *
import nidaqmx
import time, os
import numpy as np
import qcodes as qc
from src.util import connect_to_station, connect_station_instruments
from qcodes import initialise_or_create_database_at, load_by_run_spec
from qcodes import Station
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430
from qcodes.instrument_drivers.stanford_research.SR860 import SR860
from qcodes_contrib_drivers.drivers.Oxford.IPS120 import OxfordInstruments_IPS120
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.data_set import load_by_run_spec
from qcodes.dataset.data_export import get_data_by_id
from qcodes.instrument_drivers.stanford_research.SR830 import SR830

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\nanouser\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\nanouser\.qcodes\logs\210406-10128-qcodes.log


In [2]:
station = Station(config_file='cfg/default.station.yaml')

In [3]:
station.snapshot()

{'instruments': {},
 'parameters': {},
 'components': {},
 'config': {'instruments': ordereddict([('daq', ordereddict([('address', 'Dev1'), ('enable_forced_reconnect', True), ('type', 'src.daq_driver.Daq')])), ('itc', ordereddict([('address', 'GPIB0::24::INSTR'), ('enable_forced_reconnect', True), ('type', 'src.ITC503.ITC503')])), ('keith', ordereddict([('address', 'USB0::0x05E6::0x2450::04358606::INSTR'), ('enable_forced_reconnect', True), ('type', 'qcodes.instrument_drivers.tektronix.Keithley_2450.Keithley2450')])), ('ls340', ordereddict([('address', 'GPIB0::12::INSTR'), ('enable_forced_reconnect', True), ('type', 'src.Model_340.Model_340')])), ('sr860', ordereddict([('address', 'USB0::0xB506::0x2000::003594::INSTR'), ('enable_forced_reconnect', True), ('type', 'qcodes.instrument_drivers.stanford_research.SR860.SR860')]))])}}

In [4]:
station.snapshot()['config']['instruments'].items()

In [5]:
instruments = {}
for (name, other) in station.snapshot()['config']['instruments'].items():
    instruments[name] = station.load_instrument(name)

2021-04-06 13:42:22,917 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [daq_ao0(DaqAOChannel)] Snapshot: Could not update parameter: gain_factor
2021-04-06 13:42:22,928 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [daq_ao0(DaqAOChannel)] Snapshot: Could not update parameter: impedance
2021-04-06 13:42:22,929 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [daq_ao0(DaqAOChannel)] Snapshot: Could not update parameter: value
2021-04-06 13:42:23,034 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [daq_ao1(DaqAOChannel)] Snapshot: Could not update parameter: gain_factor
2021-04-06 13:42:23,036 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [daq_ao1(DaqAOChannel)] Snapshot: Could not update parameter: impedance
2021-04-06 13:42:23,040 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [daq_ao1(DaqAOChannel)] Snapshot: Could not update parameter: value
2021-04-06 13:42:23,063 ¦ qcodes

Connected to: NI DAQ USB-6259 (BNC) (Dev1) in 1.25s


2021-04-06 13:42:23,113 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [daq_ai0(DaqAIChannel)] Snapshot: Could not update parameter: gain_factor
2021-04-06 13:42:23,115 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [daq_ai0(DaqAIChannel)] Snapshot: Could not update parameter: impedance
2021-04-06 13:42:23,117 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [daq_ai0(DaqAIChannel)] Snapshot: Could not update parameter: value
2021-04-06 13:42:23,139 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [daq_ai1(DaqAIChannel)] Snapshot: Could not update parameter: gain_factor
2021-04-06 13:42:23,141 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [daq_ai1(DaqAIChannel)] Snapshot: Could not update parameter: impedance
2021-04-06 13:42:23,142 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [daq_ai1(DaqAIChannel)] Snapshot: Could not update parameter: value
2021-04-06 13:42:23,164 ¦ qcodes

Connected to: OXFORD ITC503 (serial:None, firmware:1.11) in 0.77s


2021-04-06 13:42:26,670 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [keith_source(Source2450)] Snapshot: Could not update parameter: sweep_axis


Connected to: KEITHLEY INSTRUMENTS 2450 (serial:04358606, firmware:1.6.4c) in 0.08s


2021-04-06 13:42:31,687 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [keith_source(Source2450)] Snapshot: Could not update parameter: user_delay
2021-04-06 13:42:31,719 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [keith_source(Source2450)] Snapshot: Could not update parameter: sweep_axis
2021-04-06 13:42:36,725 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [keith_source(Source2450)] Snapshot: Could not update parameter: user_delay
2021-04-06 13:42:41,745 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [keith_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
2021-04-06 13:42:46,771 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [keith_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
2021-04-06 13:42:51,792 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [keith_sense(Sense2450)] Snapshot: Could not update parameter: user_delay


Connected to: LSCI 340 (serial:000000, firmware:021901) in 0.05s


2021-04-06 13:42:52,481 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [ls340_output_1(Output_340)] Snapshot: Could not update parameter: mode
2021-04-06 13:42:52,513 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [ls340_output_1(Output_340)] Snapshot: Could not update parameter: input_channel
2021-04-06 13:42:52,535 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [ls340_output_1(Output_340)] Snapshot: Could not update parameter: units
2021-04-06 13:42:52,538 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [ls340_output_1(Output_340)] Snapshot: Could not update parameter: current_or_power
2021-04-06 13:42:52,564 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [ls340_output_1(Output_340)] Snapshot: Could not update parameter: powerup_enable
2021-04-06 13:42:52,732 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 215 ¦ [ls340_output_2(Output_340)] Snapshot: Could not update parameter: m

Connected to: Stanford_Research_Systems SR860 (serial:003594, firmware:V1.51) in 0.08s


In [6]:
print(instruments)
daq = instruments['daq']
keith = instruments['keith']
ls340 = instruments['ls340']
itc = instruments['itc']
sr860 = instruments['sr860']

{'daq': <Daq: daq>, 'itc': <ITC503: itc>, 'keith': <Keithley2450: keith>, 'ls340': <Model_340: ls340>, 'sr860': <SR860: sr860>}


In [115]:
itc.P()

80.0

In [55]:
sr860.X.label = 'I_sd'
sr860.P.label = 'P_I_sd'
keith.source.voltage.label = 'V_tg'
daq.ai0.voltage.label = 'V_12-13'
daq.ai1.voltage.label = 'P_V_12-13'
daq.ai2.voltage.label = 'V_2-6'
daq.ai3.voltage.label = 'P_V_2-6'
daq.ai4.voltage.label = 'V_6-12'
daq.ai5.voltage.label = 'P_V_6-12'

In [103]:
daq.ai2.voltage()/2.5

1.3388139665167724

In [11]:
db = os.environ['MeasureItHome']+'\\Databases\\SW2.db'

## Initialize the magnet driver

In [12]:
magnet = OxfordInstruments_IPS120('magnet', 'GPIB0::25::INSTR', use_gpib=True)

## Let's make sure we're connected and everything is as we expect

In [13]:
magnet.identify()

'IPS120-10  Version 3.07  (c) OXFORD 1996'

In [38]:
magnet.examine()

System Status: 
Normal
Activity: 
To zero
Local/Remote status: 
Local and unlocked
Switch heater: 
Off magnet at field (switch closed)
Mode: 
Tesla, Magnet sweep: fast
Polarity: 
Desired: Negative, Magnet: Negative, Commanded: Negative, Both contactors open


## Set the control status 
#### 0 : Local and locked
#### 1 : Remote and locked
#### 2 : Local and unlocked
#### 3 : Remote and unlocked

In [15]:
magnet.remote_status(3)

### We can check and set the field setpoints and the sweeprate of the field

In [ ]:
magnet.field_setpoint()

In [ ]:
magnet.field_setpoint(0)

In [ ]:
magnet.sweeprate_field()

In [ ]:
magnet.sweeprate_field(0.3)

## Check (and set) the activity status
#### 0 : Hold
#### 1 : Goto zero
#### 2 : Goto setpoint

In [ ]:
magnet.activity()

## Turn the switch heater on to enable the field to be ramped

In [ ]:
magnet.heater_on()

## Now, we can run to the field, either by calling 'run_to_field', or setting the activity to 'goto setpoint' 

In [ ]:
magnet.run_to_field(12)

## If we want to stay at field to do measurements, hold the field in place by setting persistent mode

In [ ]:
magnet.set_persistent()

#### You can check that we're in persistent mode by checking the persistent field (field in the magnet) and the field (which is the current going through the leads- should be 0)

In [ ]:
magnet.persistent_field()

In [ ]:
magnet.field()

## Once done, leave persistent mode so we can ramp the field back to zero

In [ ]:
magnet.leave_persistent_mode()

## Once we are done, we can go back to 0 either by 'run_to_field' or setting activity to 'goto zero'

In [ ]:
magnet.run_to_field(0)

## Finally, switch the heater off and set us to 'hold'

In [101]:
magnet.heater_off()
magnet.activity(0)

Setting switch heater... (wait 40s)


## Double check to make sure the field and current are 0, and it is set to hold, and you can turn the IPS power off

In [ ]:
keithley_tg.output_enabled(1)
keithley_bg.output_enabled(1)
keithley_sg.output_enabled(1)

In [ ]:
done_with_sweeps=False
print(done_with_sweeps)
def iterate():
    global done_with_sweeps
    done_with_sweeps=True

iterate()
print(done_with_sweeps)

In [ ]:
keithley_tg.source.voltage()

In [ ]:
magnet.run_to_field(0)

In [54]:
keith.sense.current()

-3.899555e-11

In [ ]:
ls340.A.temperature()

In [ ]:
sweep0 = Sweep0D(inter_delay=0.5)
sweep0.follow_param(magnet.field)

In [ ]:
sweep0.start()

In [ ]:
magnet.field.get()

In [ ]:
sweep.instrument

In [ ]:
magnet.heater_off()

In [ ]:
magnet.activity(0)

In [98]:
magnet.leave_persistent_mode()

Setting switch heater... (wait 40s)


In [37]:
magnet.set_persistent()

Setting switch heater... (wait 40s)
Waiting for the switch heater to become superconducting


In [ ]:
magnet.field_setpoint(0.1)

In [ ]:
magnet.mode2()

In [ ]:
magnet.persistent_field()

In [97]:
try:
    # Make sure database_name and the path are set to the correct values!
    db = 'Databases\\SW2.db'
    exp_name = "B sweep 6-0T"
    sample_name = "Hall_V9-14=1mVac_T=4K_Vbg=0V_gain=1e8_meas 2-6 12-13 6-12"
    initialise_or_create_database_at(db)
    qc.new_experiment(name=exp_name, sample_name=sample_name)
except:
    print("Error opening database")

In [99]:
sweep = SweepIPS(magnet, 0, inter_delay=0.25, save_data=True, persistent_magnet=False, plot_data = True)
sweep.follow_param(sr860.X, sr860.P, keith.sense.current, daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage)

In [100]:
sweep.start()

Starting experimental run with id: 81. 
Checking the status of the magnet and switch heater.
Heater is already on, so the magnet was not in persistent mode
Done with the sweep, t=1201.780999999959 s
Done with the sweep!


In [ ]:
sweep.plotter.data_queue

In [ ]:
sweep.stop()

In [43]:
sweep.kill()

In [104]:
sweep_gate = Sweep1D(keith.source.voltage, -8, 0, .05, inter_delay=0.25, bidirectional=False, save_data=False, plot_data=True, x_axis_time=False)
sweep_gate.follow_param(sr860.X, sr860.P, keith.sense.current, daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage)

In [105]:
sweep_gate.start()

Ramping to our starting setpoint value of -8 V
Done ramping V_tg to -8
Sweeping V_tg to 0 V


In [106]:
sweep_gate.is_running

False

In [107]:
sweep_gate.kill()

In [94]:
daq.ai6.voltage()

1.7027949103388373

In [ ]:
ramp_sg = Sweep1D(keithley_sg.source.voltage, 0, 10, -.1, inter_delay=0.25, save_data=False, x_axis_time = False)
ramp_sg.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)

In [ ]:
ramp_sg.start()

In [ ]:
sweep_queue = SweepQueue(inter_delay = 1)
exp_name = '210203-Landau_fan_unknown_temp_Vsg=10V'

Vtg_min, Vtg_max = -8., 8.
Vtg_step_slow = 0.05
Vtg_step_fast = 0.2

Vbg_min, Vbg_max = -13., 13.
Vbg_step_slow = 0.08
Vbg_step_fast = 0.3

inter_delay_slow = 0.5
inter_delay_fast = 0.25

N = 60 #data points
B_start, B_end = 2.5, 6.25
B_crit = 5

for B in np.linspace(B_start, B_end, N):
    #Magnet sweep
    if B>=B_crit:
        persistent_magnet = True
    else:
        persistent_magnet = False
    mag_sweep = SweepIPS(magnet, B, inter_delay=0.25, save_data=False, persistent_magnet=persistent_magnet)
    #mag_sweep.follow_param(ls340.A.temperature)
    
    # Create the database entry
    sample_name = f'B={B:.2f}T_sweep-mag'
    db_entry = DatabaseEntry(db, exp_name, sample_name)
    sweep_queue.append(db_entry)
    sweep_queue.append(mag_sweep)
    
    # TG sweep
    tg_initialize = Sweep1D(keithley_tg.source.voltage, 0, Vtg_min, Vtg_step_fast, inter_delay=inter_delay_fast, x_axis_time=0, save_data = False)
    tg_sweep = Sweep1D(keithley_tg.source.voltage, Vtg_min, Vtg_max, Vtg_step_slow, inter_delay=inter_delay_slow, x_axis_time=0, save_data = True)
    tg_rampdown = Sweep1D(keithley_tg.source.voltage, Vtg_max, 0, Vtg_step_fast, inter_delay=inter_delay_fast, x_axis_time=0, save_data = False)
    
    tg_sweep.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
    
    # Create the database entry
    sample_name = f'B={B:.2f}T_Vbg=0V_sweep-Vtg'
    db_entry = DatabaseEntry(db, exp_name, sample_name)
    sweep_queue.append(db_entry)
    sweep_queue.append(tg_initialize, tg_sweep, tg_rampdown)

    bg_initialize = Sweep1D(keithley_bg.source.voltage, 0, Vbg_min, Vbg_step_fast, inter_delay=inter_delay_fast, x_axis_time=0, save_data = False)
    bg_sweep = Sweep1D(keithley_bg.source.voltage, Vbg_min, Vbg_max, Vbg_step_slow, inter_delay=inter_delay_slow, x_axis_time=0, save_data = True)
    bg_rampdown = Sweep1D(keithley_bg.source.voltage, Vbg_max, 0, Vbg_step_fast, inter_delay=inter_delay_fast, x_axis_time=0, save_data = False)
    
    bg_sweep.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
    
    # Create the database entry
    sample_name = f'B={B:.2f}T_Vtg=0V_sweep-Vbg'
    db_entry = DatabaseEntry(db, exp_name, sample_name)
    sweep_queue.append(db_entry)
    sweep_queue.append(bg_initialize, bg_sweep, bg_rampdown)
    


In [ ]:
for item in sweep_queue.queue:
    print(item)

In [ ]:
sweep_queue.start()

In [ ]:
sweep_queue.current_sweep

In [ ]:
print(sweep_queue.current_sweep)

In [ ]:
sweep.update_values()

In [ ]:
sweep_queue.current_sweep

In [ ]:
sweep.runner

In [ ]:
sweep.save_data

In [ ]:
sweep_queue.current_sweep.stop()

In [ ]:
magnet.activity(0)

In [ ]:
sweep_queue.queue